In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, average_precision_score, cohen_kappa_score

import ml.ML_util as ML_util

In [2]:
PATH_DF = 'ml/results_rf_310322_chance_balanced.csv'
PATH_PRED = 'ml/predictions/preds_rf_310322_chance_balanced.npy'
PATH_DATA = 'data_sets_310322_chance_balanced'

In [13]:
PATH_DATA2 = 'data_sets_290322/complete_0/morphological/0_0.800.2/'
train, _, _, _, _, _ = ML_util.get_dataset(PATH_DATA2)
train = ML_util.squeeze_clusters(train)
print(train[:, 1])

[37. 37. 36. 37. 31. 34. 36. 29. 33. 34. 30. 40. 31. 38. 35. 42. 32. 39.
 47. 36. 31. 37. 41. 32. 30. 41. 38. 33. 30. 36. 32. 39. 56. 35. 36. 38.
 34. 28. 38. 35. 33. 49. 37. 31. 35. 30. 34. 39. 37. 33. 37. 37. 38. 36.
 33. 34. 36. 40. 34. 36. 30. 45. 33. 39. 38. 37. 33. 31. 41. 38. 33. 37.
 34. 46. 35. 30. 33. 35. 39. 38. 37. 31. 42. 31. 35. 31. 34. 38. 34. 36.
 37. 43. 33. 32. 34. 39. 36. 30. 33. 28. 35. 39. 33. 30. 31. 27. 32. 36.
 37. 32. 39. 30. 34. 33. 39. 33. 40. 30. 35. 39. 36. 31. 46. 36. 29. 52.
 28. 32. 30. 32. 38. 36. 37. 30. 30. 36. 36. 35. 36. 38. 36. 30. 40. 37.
 33. 28. 37. 30. 29. 38. 33. 38. 33. 32. 31. 36. 40. 29. 28. 34. 35. 37.
 39. 32. 37. 40. 42. 34. 35. 33. 32. 32. 32. 29. 35. 33. 39. 32. 33. 40.
 33. 31. 33. 34. 34. 46. 32. 29. 35. 34. 32. 36. 33. 31. 32. 33. 41. 38.
 38. 37. 32. 39. 28. 34. 26. 38. 30. 30. 26. 31. 39. 38. 36. 23. 28. 35.
 33. 29. 34. 32. 36. 30. 30. 35. 29. 32. 36. 36. 35. 33. 38. 32. 39. 34.
 35. 30. 32. 37. 33. 32. 41. 38. 30. 33. 29. 30. 34

In [3]:
def get_labels():
    labels = []
    for i in range(50):  # 50=number of seeds
        data_path = PATH_DATA + f'/complete_{i}/spatial/0_0.800.2/'
        _, _, test, _, _, _ = ML_util.get_dataset(data_path)
        test_squeezed = ML_util.squeeze_clusters(test)
        _, y = ML_util.split_features(test_squeezed)
        labels.append(y)
    labels = np.repeat(labels, 1 * 3, axis=0)  # 8=number of chunk sizes; 3 number of modalities
    return labels

In [4]:
results = pd.read_csv(PATH_DF, index_col=0)
complete = results[results.restriction == 'complete']
complete = complete.dropna(how='all', axis=1)

preds = np.load(PATH_PRED)

labels = get_labels()

In [5]:
auc_flipped = [roc_auc_score(1-label, 1-pred) for label, pred in zip(labels, preds)]
ap = [average_precision_score(label, pred) for label, pred in zip(labels, preds)]
ap_flipped = [average_precision_score(1-label, 1-pred) for label, pred in zip(labels, preds)]
kappa = [cohen_kappa_score(label, pred>=0.5) for label, pred in zip(labels, preds)]

In [6]:
complete['auc_flipped'] = auc_flipped
complete['kappa'] = kappa
complete['ap'] = ap
complete['ap_flipped'] = ap_flipped

In [7]:
grouped_complete = complete.groupby(by=['restriction', 'modality', 'chunk_size'])
grouped_complete[['auc', 'auc_flipped', 'ap', 'ap_flipped']].median()

auc  auc_flipped        ap  \
restriction modality      chunk_size                                    
complete    morphological 0.0         0.464429     0.464429  0.787938   
            spatial       0.0         0.483362     0.483362  0.795945   
            temporal      0.0         0.461130     0.461130  0.790013   

                                      ap_flipped  
restriction modality      chunk_size              
complete    morphological 0.0           0.223589  
            spatial       0.0           0.201328  
            temporal      0.0           0.214193